In [19]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.neighbors import NearestNeighbors
import requests
import json
import os
import re
from sklearn.metrics import f1_score as sklearn_f1_score
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple, Dict
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# load data set
def load_local_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

train_data = load_local_data('/Users/wangchenwei/Downloads/TLQA_NLP_Porject/data/train_TLQA.json')
test_data = load_local_data('/Users/wangchenwei/Downloads/TLQA_NLP_Porject/data/test_TLQA.json')
val_data = load_local_data('/Users/wangchenwei/Downloads/TLQA_NLP_Porject/data/val_TLQA.json')
train_questions = [item["question"] for item in train_data]
train_answers = [item["final_answers"] for item in train_data]
val_questions = [item["question"] for item in val_data]
test_questions = [item["question"] for item in test_data]

In [32]:
# Fit KNN on the training embeddings
knn = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(train_embeddings)

# For each test question, find the closest training questions
few_shot_examples = []
for val_embedding in val_embeddings:
    distances, indices = knn.kneighbors([val_embedding])
    few_shot_samples = [{'question': train_questions[idx], 'answers': train_answers[idx]} for idx in indices[0]]
    few_shot_examples.append(few_shot_samples)

In [4]:
def knn_search(train_questions, train_answers, test_questions, k=5, model_name='sentence-transformers/all-mpnet-base-v2'):
  
    model = SentenceTransformer(model_name)
    
   
    train_embeddings = model.encode(train_questions, convert_to_tensor=True).cpu().numpy()
    
    test_embeddings = model.encode(test_questions, convert_to_tensor=True).cpu().numpy()
    
    #嵌入上拟合KNN模型
    knn = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(train_embeddings)
    
    # 为每个验证问题找到最近的训练问题
    few_shot_examples = []
    for test_embedding in test_embeddings:
        distances, indices = knn.kneighbors([test_embedding])
        few_shot_samples = [{'question': train_questions[idx], 'answers': train_answers[idx]} for idx in indices[0]]
        few_shot_examples.append(few_shot_samples)
    
    return few_shot_examples

In [110]:
# Run generative models
def run_model(model_name, test_questions, few_shot_examples, repetition_penalty=3.0):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    predictions = []
    for test_question, examples in zip(test_questions, few_shot_examples):
        context = "\n".join([f"Q: {ex['question']} A: {', '.join(ex['answers'])}" for ex in examples])
        input_text = f"{context}\nQ: {test_question} A:"
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    
        # Generate prediction with repetition penalty
        outputs = model.generate(**inputs, max_new_tokens=100, repetition_penalty=repetition_penalty)
    
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        prediction = prediction.replace("A: ", "").strip()
    
        predictions.append(prediction)
        print(f"Question: {test_question}\nGenerated Answer: {prediction}\n")

    return predictions

In [97]:
#flan-t5-large
# 3 shot
examples_3shot  = knn_search(train_questions, train_answers, test_questions, 3)
predictions_3shot = run_model("google/flan-t5-large", test_questions, examples_3shot)

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: Manchester City F.C. (2010–11), Birmingham City F.C. (2011–12), Bristol Rovers F.C. (2012–13), Leeds United F.C. (2013–15), Sheffield Wednesday F.C. (2015–16)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Ukraine (2010–2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: Teachta Dála (2010, 2011, 2012, 2013, 2014), Social Democratic Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Generated Answer: Manchester City F.C. (2010, 2011), Birmingham City F.C. (2011, 2012, 2014), Newcastle United F.C. (2012, 2014, 2015, 2016)

Question: List all heads of the governmen

Question: List all political parties Lucy Gichuhi, also known as Lucy Muringo Gichuhi, was a member of from 2010 to 2020.
Generated Answer: Democratic Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2020)

Question: List all sports teams Vincenzo Nibali, also known as Lo Squalo, played for from 2010 to 2020.
Generated Answer: Italy national football team (2010, 2011, 2012, 2014, 2015), Italian national basketball team (2012, 2013), Italian national volleyball team (2013, 2014, 2015), Italian national ice hockey team (2011, 2012, 2013), Italian national handball team (2009, 2010), Italian national handball team (2010, 2011), Italian national handball team (2010, 2011), Italian national handball team (2010, 2011), Italian national handball team (2010, 2011), Italian national hand

Question: List all sports teams Sam Tomkins played for from 2010 to 2020.
Generated Answer: Birmingham City F.C.

Question: List all sports teams Juan Cuadrado, also known as Juan Guillermo Cuadrado Bello, pla

Question: List all coaches of Vitória S.C., also known as Vitória de Guimâraes, from 2015 to 2020
Generated Answer: José Lus Rodrguez (2015–2020), Jorge Alves (2016–2020), Pedro de Almeida (2017–2020), Joo Ribeiro (2019–2020)

Question: List all coaches of U.C. Trevigiani Energiapura Marchiol, also known as Unieuro Trevigiani-Hemus 1896, from 2012 to 2020
Generated Answer: Antonio Giancarlo (2012–2020), Maurizio Sforza (2013–2015), Roberto De Luca (2014–2015), Francesco Ricciardo (2016–2017), Paolo Ferreri (2018–2019), Marco Alessandro (2019), Andreas Salerno (2019), Stefano Gatti (2019), Carlo Cacioppo (2019), Mario Mand

Question: List all coaches of Silkeborg IF, also known as Silkeborg Idrætsforening, from 2015 to 2020
Generated Answer: Hans-Joachim hlund (2015–2020), Lars Bjrnsson (2016–2020), Johan Svendsen (2017–2019), Anders Lindegaard (2018–2019), Erik Kjellberg (2019–2020), Jordi Ljungberg (2019–2020), Jens Timmerman (2019–2020), Henning Berger (2019–2020), Jan Stensen (2019–

Question: List all employers Michael Hout, also known as Mike Hout, worked for from 2010 to 2020.
Generated Answer: He was a member of the board of directors for the New York Yankees, Philadelphia Phillies, and Los Angeles Dodgers.

Question: List all heads of the government of Guinea, also known as Guinea-Conakry, from 2010 to 2020 
Generated Answer: Emmanuel Ayoto, Issa Amin, Samuel Omar, Yahya Jammeh

Question: List all employers Linda Bauld, also known as Linda C. Bauld, worked for from 2014 to 2020.
Generated Answer: United States Department of Agriculture (USDA), University of California, Davis

Question: List all coaches of PSV Eindhoven, also known as Philips Sport Vereniging, from 2013 to 2020
Generated Answer: David van der Merwe (2013–2020), Jan Vermeer (2012–2016), Jordi Rietveld (2015–2016), Jaap Stam (2017–2018), Joost van den Berg (2019–2020), Henri Mats Hummels (2019–2020)

Question: List all employers Pedro J. Ramírez worked for from 2010 to 2020.
Generated Answer: El 

Question: List all employers Kristen L. Knutson, also known as Kristen Knutson, worked for from 2010 to 2020.
Generated Answer: University of California, Berkeley

Question: List all educational institutions Tyler Dorsey, also known as Tyler Quincy Dorsey, attended from 2011 to 2017.
Generated Answer: University of Oklahoma

Question: List all employers Carenza Lewis worked for from 2010 to 2020.
Generated Answer: The New York Yankees (2010, 2011, 2012, 2014, 2015, 2016), the Los Angeles Dodgers (2010, 2011, 2012, 2014, 2015, 2016), the Chicago White Sox (2010, 2011, 2012, 2014, 2015, 2016), the Seattle Seahawks (2010, 2011, 2012, 2014, 2015, 2016), the Portland Trail Blazers (2010, 2011, 2012, 2014, 2015, 2016), the San Francisco Giants (2010, 2011, 2012, 2014,

Question: List all coaches of Louisville City FC, also known as Lou City, from 2014 to 2020
Generated Answer: Michael O'Neill (2014, 2015), Mikel Arteta (2015, 2016), Mark McCormack (2016, 2017), David McMillan (2017, 2018), K

...

Question: List all positions Stockwell Day, also known as Stockwell Burt Day, held from 2010 to 2011.
Generated Answer: United States Representative (2010, 2011)

Question: List all coaches of RCD Espanyol de Barcelona, also known as RCDE, from 2010 to 2020
Generated Answer: José Mara Luis Figo (2010, 2011), Josep Maria Bartomeu (2012, 2013), Jordi Alba (2016, 2017), Javier Mascherano (2017, 2018), Juan Carlos Fernández (2018, 2019), Joao Moutinho (2019), Diego Simeone (2020), Francisco López-Alvarez (2019), Andrés Iniesta (2019), Pedro Alvarado

Question: List all chairpersons of Social Democrats, also known as Socialdemokratiet, from 2010 to 2020
Generated Answer: Andreas Fogh Rasmussen (2010, 2011), Anders Behring Breivik (2011, 2012), Lars-Göran ylund (2012, 2014), Jens Stoltenberg (2014, 2015), Johan Svendsen (2015, 2016), Janusz Skorzenowski (2016, 2017), Thomas J. Olofsson (2017, 2018)

Question: List all positions Jyrki Katainen, also known as Jyrki Tapani Katainen, held from 2

Question: List all employers Matt Ouimet, also known as Matthew Anthony Ouimet, worked for from 2010 to 2020.
Generated Answer: United States Navy (2010, 2011, 2012, 2014, 2015, 2016)

Question: List all employers Hunter Biden, also known as Robert Hunter Biden, worked for from 2013 to 2020.
Generated Answer: United States Department of State, New York City

Question: List all sports teams Adil Rami played for from 2010 to 2020.
Generated Answer: South Australia cricket team (2010, 2011, 2012), Adelaide Strikers (2015, 2016), England cricket team (2015, 2016), Durdanto Dhaka (2017, 2018, 2019, 2020)

Question: List all chairpersons of Cabinet of Ministers of Ukraine, also known as Government of Ukraine, from 2010 to 2020
Generated Answer: Kyrgyzstan's Yuri Miloevi, Ukraine's Minister of Foreign Affairs, and the President of Ukraine

Question: List all chairpersons of TOP 09, also known as Tradition Responsibility Prosperity, from 2010 to 2020
Generated Answer: Michael O'Neill (2010, 20

Question: List all sports teams Humam Tariq, also known as Humam Tariq Faraj Naoush, played for from 2010 to 2020.
Generated Answer: Pakistan national football team (2010, 2011, 2012, 2014), Qatar national football team (2012, 2013), Bahrain national football team (2013, 2014), Saudi Arabia national football team (2014, 2015), United Arab Emirates national football team (2015, 2016),

Question: List all employers Julia Y Chan, also known as Julia Y. Chan, worked for from 2010 to 2020.
Generated Answer: Yokohama Electric Power Corporation (2010, 2011), Fujitsu Heavy Industries (2011, 2012)

Question: List all sports teams Peter Crouch, also known as Peter James Crouch, played for from 2010 to 2020.
Generated Answer: England national under-21 football team (2010, 2011), Manchester United F.C. (2011, 2012, 2014), Southampton F.C. (2012, 2013), West Ham United F.C. (2013, 2014, 2015)

Question: List all educational institutions Gary Payton II, also known as Gary Dwayne Payton II, attended 

Question: List all sports teams Filipe Morais, also known as Filipe Alexandre Major Morais, played for from 2010 to 2020.
Generated Answer: Brazilian footballer Filipe Morais played for the following clubs from 2010 to 2020:

Question: List all political parties Marina Weisband was a member of from 2010 to 2020.
Generated Answer: The Greens (2010, 2011, 2012, 2014), Labour Party (2011, 2012, 2014), Liberal Democrats (2012, 2014, 2015), Socialist Party of Catalonia (2013, 2014, 2015), Independent (2014, 2015, 2016, 2017, 2018)

Question: List all coaches of Spain women's national association football team, also known as Spain women's national soccer team, from 2010 to 2020
Generated Answer: José Manuel Barroso (2010, 2011), lvaro de la Rosa (2012, 2013), Mara Elena Fernández (2011, 2012), Javier Mascherano (2013, 2014), Luisa Valverde (2016, 2017), Jordi Alba (2017, 2018), Joo Garca (2018, 2019), Juan Carlos Martnez (2019)

Question: List all educational institutions Shawné Michaelain H

Question: List all coaches of Inteja-Dominican, also known as Inteja-MMR Dominican, from 2015 to 2020
Generated Answer: Antonio Aguirre (2015–2020), José Luis Rodrguez (2016–2020), Juan Carlos Fernández (2017–2020), Jose Mara del Rosario (2019–2020), Francisco Martnez (2019–2020), Xavier Ramos (2019–2020), Miguel Alvarez (2019–2020), Pedro Ayala (2019–2020), Rafael López-Alvarado (2019–

Question: List all sports teams Jonny Howson, also known as Jonathan Mark Howson, played for from 2010 to 2020.
Generated Answer: Manchester City F.C. (2010, 2011, 2012, 2014, 2015), Birmingham City F.C. (2012, 2013), Leeds United F.C. (2013, 2014, 2015), Sheffield Wednesday F.C. (2014, 2015), Bradford City F.C. (2015, 2016), Middlesbrough F.C. (2016, 2017), Wigan Athletic F.C. (2017, 2018)

Question: List all entities that owned MyAnimeList, also known as MyAnimeList, LLC, from 2010 to 2019.
Generated Answer: MyAnimeList, LLC

Question: List all political parties Sarath Fonseka was a member of from 20

In [102]:
# 5 shot
#examples_5shot  = knn_search(train_questions, train_answers, test_questions, 5)
predictions_5shot_flanlarge = run_model("google/flan-t5-large", test_questions, examples_5shot)

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: Manchester City F.C. (2010, 2011), Birmingham City F.C. (2011, 2012, 2014), Leeds United F.C. (2012, 2013), Sheffield Wednesday F.C. (2013, 2014), Bradford City F.C. (2014, 2015)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Russia (2010–2020), Minister of Foreign Affairs (2012–2016), Prime Minister of Russia (2015–2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: Teachta Dála (2010, 2011, 2012, 2013, 2014), Independent (2011, 2012, 2013, 2014), Social Democratic Party (2010, 2011, 2012, 2013, 2014), Green Party (2010, 2011, 2012, 2013, 2014), Labour Party (2010, 2011, 2012, 2013, 2014), Liberal Party (2010, 2011, 2012, 2013, 2014), Conservative Party (2010, 2011, 2012, 2013,

Question: List all political parties Lucy Gichuhi, also known as Lucy Muringo Gichuhi, was a member of from 2010 to 2020.
Generated Answer: Democratic Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2020), Greens

Question: List all sports teams Vincenzo Nibali, also known as Lo Squalo, played for from 2010 to 2020.
Generated Answer: Italy national football team (2010, 2011, 2012, 2014), Italian national basketball team (2012, 2013), Italy national volleyball team (2013, 2014), Italy national women's volleyball team (2014, 2015)

Question: List all sports teams Sam Tomkins played for from 2010 to 2020.
Generated Answer: Birmingham City F.C. (2010, 2011), Manchester United F.C. (2011, 2012), Newcastle United F.C. (2012, 2013), Leeds Rhinos F.C. (2013, 2014), Bradford City F.C. (2014, 2015)

Question: List all sports teams Juan Cuadrado, also known as Juan Guillermo Cuadrado Bello, played for from 2010 to 2020.
Generated Answer: Chelsea F.C.

Question: List all political parties Rachel 

Question: List all sports teams Nick Swisher, also known as Nicolas Thompson Swisher, played for from 2010 to 2020.
Generated Answer: New York Rangers (2010, 2011), Los Angeles Clippers (2011, 2012), Boston Celtics (2012, 2013), Philadelphia Flyers (2013, 2014), Chicago Cubs (2014, 2015)

Question: List all sports teams Darwin Quintero, also known as Carlos Quintero, played for from 2010 to 2020.
Generated Answer: Santos FC (2010, 2011, 2012, 2014), Real Sociedad (2011, 2012, 2014, 2015, 2016, 2017), Club Atlético Madrid (2019, 2020)

Question: List all positions François Lecointre, also known as General François Lecointre, held from 2016 to 2020.
Generated Answer: General François Lecointre (born 16 April 1980) is a French general and politician.

Question: List all coaches of Vitória S.C., also known as Vitória de Guimâraes, from 2015 to 2020
Generated Answer: José Lus Rodrguez (2015–2020), Jorge Alves (2016–2020), Pedro de Almeida (2017–2020), Joo Ribeiro (2018–2019), Carlos Velasco

Question: List all employers Peter Scholze worked for from 2011 to 2020.
Generated Answer: AB InBev (2011-2020), Deutsche Telekom AG (2012-2016), ABB Group (2015-2016), Siemens SA (2017-2018), TNT GmbH (2018-2019), STMicroelectronics Ltd. (2019-2021), and BNP Paribas SA (2020-present).

Question: List all employers Susanne Ås Sivborg worked for from 2010 to 2020.
Generated Answer: AB Inbev (2010, 2011), ABN AMRO (2011, 2012), BNP Paribas (2012, 2014), RWE Group (2014, 2015)

Question: List all employers Ann Sarnoff, also known as Ann M. Sarnoff, worked for from 2010 to 2019.
Generated Answer: University of California, Berkeley (2010, 2011), Stanford Graduate School of Education (2011, 2012, 2014), Harvard Business School (2012, 2013), Columbia Law School (2013, 2015)

Question: List all employers Michael Hout, also known as Mike Hout, worked for from 2010 to 2020.
Generated Answer: United States Navy (2010, 2011), USS "Bobby" (1911), SS "Caspian" (1912), USS "Alcatraz" (1913), USS "Mid

Question: List all chairpersons of Ministry of Internal Affairs of Ukraine, also known as Ministry of Internal Affairs, from 2010 to 2020
Generated Answer: Mikhail Khrushchev (2010, 2011), Oleksandr Kostunica (2011, 2012), Sergey Yatseniuk (2012, 2014, 2015, 2016)

Question: List all heads of the government of Hamburg, also known as Free and Hanseatic City of Hamburg, from 2010 to 2020 
Generated Answer: Michael Schäuble (2010, 2011, 2012, 2014, 2015, 2020)

Question: List all positions Kevin Cramer, also known as Kevin John Cramer, held from 2013 to 2020.
Generated Answer: chief investment officer (CIO)

Question: List all positions Viktor Yanukovych, also known as Viktor Yanukovich, held from 2010 to 2014.
Generated Answer: President of Ukraine (2010–2014)

Question: List all political parties Jordi Turull i Negre, also known as Jordi Turull, was a member of from 2010 to 2020.
Generated Answer: Jordi Turull i Negre (2010, 2011, 2012, 2013, 2014, 2015), Catalan European Democratic Par

...

Question: List all positions Bob Filner, also known as Robert Earl "Bob" Filner, held from 2010 to 2013.
Generated Answer: United States Representative (2010, 2011), United States Senator (2011, 2012)

Question: List all educational institutions Filip Petrušev attended from 2016 to 2020.
Generated Answer: University of Tbilisi (2016, 2017), Faculty of Law, National Higher School of Economics (2017, 2018),

Question: List all positions Alexander De Croo held from 2014 to 2020.
Generated Answer: President of the Republic of Ireland (2014–2020), Minister for Foreign Affairs (2015–2016), Secretary of State for Trade and Industry (2017–2020), Member of Parliament (2011–2012)

Question: List all coaches of FC Nitra from 2011 to 2020
Generated Answer: Nikas efi (2011-2020), Ivan Koval (2012-2016), Jakub Szczepaski (2015-2016), Milo Forman (2017-2018), Aleksandar Zrvenkov (2018-2019), Vladimir Shvetkov (2019-2020), Nikolay Dmitriev (2019-2020), Sergey Zamryuk (2019-2020), Alexis Todoro

Questi

Question: List all employers Gregory Fu, also known as Greg Fu, worked for from 2010 to 2020.
Generated Answer: NBC Universal (2010, 2011, 2012, 2014, 2015, 2016), Fox Sports Network (2011, 2012, 2013), ESPN (2012, 2013, 2014, 2015, 2016), CBS News (2013, 2014, 2015, 2016), ABC Family (2009, 2010), Discovery Communications (2008, 2009, 2010, 2011), Disney (2007, 2008, 2009, 2010), HBO (2006, 2007, 2010), Showtime (2005, 2006, 2010), Amazon.com (2004, 2005, 2006,

Question: List all positions Noel Dempsey held from 2010 to 2011.
Generated Answer: Noel Dempsey (2010, 2011)

Question: List all positions Mahinda Rajapaksa, also known as Percy Mahendra Rajapaksa, held from 2010 to 2018.
Generated Answer: President of Sri Lanka (2010–2018), Minister of Foreign Affairs (2012–2016), Prime Minister of Sri Lanka (2015–2017)

Question: List all employers Peggy Johnson worked for from 2014 to 2020.
Generated Answer: University of California, Berkeley (2014-2020), Stanford Graduate School of Educat

Question: List all positions Vickie Chapman, also known as Vickie Ann Chapman, held from 2018 to 2020.
Generated Answer: Vice President of the United States (2018–2020), Secretary of State for Health and Human Services (2019–2020)

Question: List all coaches of Sporting Gijón, also known as Sporting Gijon, from 2017 to 2020
Generated Answer: José Luis Rodrguez (2017–2020), Juan Carlos Valverde (2019–2020), Jose Antonio Garca (2019–2020), Xavi Martnez (2019–2020), Javier Mascherano (2019–2020), Jordi Alba (2019–2020), Francisco López (2019–2020), Miguel Angel Fernández (2019–2020), Diego Marn (2019–2020), Jorge Aguil

Question: List all sports teams MS Dhoni, also known as Mahi, played for from 2010 to 2020.
Generated Answer: India national cricket team (2010–2020), Mumbai Indians (2011, 2012, 2014, 2015, 2016), Chennai Super Kings (2012, 2013), Kolkata Knight Riders (2013, 2014, 2015, 2016), Pune Warriors (2014, 2015, 2016), Rajasthan Royals (2015, 2016), Gujarat Lions (2016, 2017), Pu

Question: List all political parties Seiji Maehara was a member of from 2010 to 2020.
Generated Answer: Independent politician (2010, 2011, 2012, 2014, 2015, 2016, 2020)

Question: List all heads of the government of Myanmar, also known as Burma, from 2010 to 2020 
Generated Answer: Burma's first president, Aung San Suu Kyi (2010–2020), Myint Thani (2012–2016), Yang Hlaing (2015–2016), Thaksin Shinawatra (2016–2020)

Question: List all sports teams David Healy, also known as David Jonathan Healy, played for from 2010 to 2013.
Generated Answer: Manchester City F.C. (2010, 2011), Birmingham City F.C. (2011, 2012), Bristol Rovers F.C. (2012, 2013), Leeds United F.C. (2013, 2014)

Question: List all educational institutions Prince Joseph Wenzel of Liechtenstein, also known as Joseph Wenzel Maximilian Maria Prinz von und zu Liechtenstein, attended from 2010 to 2017.
Generated Answer: University of Liechtenstein

Question: List all political parties Muriel Ressiguier was a member of from 201

KeyboardInterrupt: 

In [101]:
# 7 shot
#examples_7shot  = knn_search(train_questions, train_answers, test_questions, 7)
predictions_7shot_flanlarge = run_model("google/flan-t5-large", test_questions, examples_7shot)

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: Middlesbrough F.C.

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Republika Srpska (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018), list of presidents of Republika Srpska (2010), Chairman of the Presidency of Bosnia and Herzegovina (2018, 2019, 2020)



KeyboardInterrupt: 

In [82]:
predictions_7shot_flanlarge = run_model("google/flan-t5-large", test_questions, examples_7shot)

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: Middlesbrough F.C.

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Republika Srpska (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018), list of presidents of Republika Srpska (2010), Chairman of the Presidency of Bosnia and Herzegovina (2018, 2019, 2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: Teachta Dála (2010, 2011, 2012, 2013, 2014), Independent (2011, 2012, 2013, 2014, 2015, 2016, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Generated Answer: New England Patriots

Question: List all heads of the government of Newfoundland and Labrador, also known as NL, from 2013 to 2020 
Generated Answer: D

Question: List all sports teams Vincenzo Nibali, also known as Lo Squalo, played for from 2010 to 2020.
Generated Answer: Real Madrid CF

Question: List all sports teams Sam Tomkins played for from 2010 to 2020.
Generated Answer: Manchester City F.C. (2010, 2011), Birmingham City F.C. (2011, 2012), Newcastle United F.C. (2012, 2013), Leeds Rhinos F.C. (2013, 2014), Bradford City F.C. (2014, 2015)

Question: List all sports teams Juan Cuadrado, also known as Juan Guillermo Cuadrado Bello, played for from 2010 to 2020.
Generated Answer: Chelsea F.C.

Question: List all political parties Rachel Carling-Jenkins, also known as Rachel Carling, was a member of from 2010 to 2018.
Generated Answer: Conservative Party (2010, 2012, 2014), Liberal Party (2012, 2013), Green Party (2013, 2014), Labour Party (2011, 2012, 2014), Independent (2009, 2010), Social Democratic Party (2008, 2009, 2010), United Kingdom Independence Party (2007, 2008, 2010), UKIP (2006, 2007), Scottish National Party (2005, 2

Question: List all employers Eric Jagielo, also known as Jagielo, Eric Stephan, worked for from 2010 to 2020.
Generated Answer: United States Navy (2010, 2011), USS "Bobby" (1911), USS "Clark" (1912), USS "David" (1913), USS "Wilson" (1914), USS "Little John" (1915), USS "Murray" (1916), USS "Henry" (1917), USS "Franklin" (1918), USS "Richardson" (19

Question: List all sports teams Nick Swisher, also known as Nicolas Thompson Swisher, played for from 2010 to 2020.
Generated Answer: Al Jazira Club (2013, 2014), Olympiacos F.

Question: List all sports teams Darwin Quintero, also known as Carlos Quintero, played for from 2010 to 2020.
Generated Answer: FC Cluj (2010, 2011, 2017, 2018, 2019), Galatasaray F.C. (2011, 2012), Orduspor (2011, 2012)

Question: List all positions François Lecointre, also known as General François Lecointre, held from 2016 to 2020.
Generated Answer: General François Lecointre (born 16 April 1981) is a French former general and politician.

Question: List all co

Question: List all entities that owned VK, also known as vk.com, from 2010 to 2014.
Generated Answer: VK Entertainment (2010, 2011), vk.com

Question: List all political parties Yuriko Koike, also known as Koike Yuriko, was a member of from 2010 to 2020.
Generated Answer: Japan People's Democratic Party (2010, 2011, 2012), Liberal Democratic Party (2011, 2012, 2014, 2015, 2016, 2020)

Question: List all coaches of Sporting CP, also known as Sporting Club Portugal, from 2018 to 2020
Generated Answer: Q: List all coaches of Valencia CF, also known as Valencia Club de Ftbol, from 2012 to 2020

Question: List all employers Peter Scholze worked for from 2011 to 2020.
Generated Answer: AB InBev (2011, 2012, 2014), Deutsche Telekom AG (2012, 2013), ABB Group (2013, 2015)

Question: List all employers Susanne Ås Sivborg worked for from 2010 to 2020.
Generated Answer: AB Inbev (2010, 2011), ABN AMRO (2011, 2012), BNP Paribas (2012, 2014), RWE Group (2014, 2015), Sveriges Bank (2015, 2016), KNVB

Question: List all political parties Andris Bērziņš, also known as Andris Berzins, was a member of from 2010 to 2020.
Generated Answer: Democratic Party of Montenegro (2010, 2011, 2012, 2014, 2015), Socialist People's Party of Montenegro (2010, 2011, 2012, 2014, 2015), National Union for Democracy (2010, 2011, 2012, 2014, 2015, 2016), Independent Party of Montenegro (2010, 2011, 2012, 2014, 2015, 2016), Republican Party of Montenegro (2010, 2011, 2012, 2014, 2015, 2016, 2020)

Question: List all chairpersons of Ministry of Internal Affairs of Ukraine, also known as Ministry of Internal Affairs, from 2010 to 2020
Generated Answer: Mikhail Khrushchev (2010, 2011), Oleksandr Kostunica (2010, 2011), Ivan Shyrokov (2010, 2011), Sergei Tymoshenko (2010, 2011), Andriy Yatsenyuk (2010, 2011), Alexei Luzhin (2010, 2011), Alexander Zhukov (2010, 2011), Oleksandr Kostunica (2010, 2011), Oleks

Question: List all heads of the government of Hamburg, also known as Free and Hanseatic City of Hamburg,

...

Question: List all employers Mary E. Northridge, also known as Mary E Northridge, worked for from 2010 to 2020.
Generated Answer: University of California, Berkeley (2010, 2011), Stanford Graduate School of Education (2011, 2012), Harvard Business School (2012, 2013), Columbia Law School (2013, 2014)

Question: List all coaches of FCI Levadia Tallinn, also known as FC Levadia Tallinn, from 2012 to 2020
Generated Answer: Levadia Tallinn (2012–2020)

Question: List all chairpersons of Vetëvendosje!, also known as Vetëvendosje, from 2010 to 2020
Generated Answer: Johan hlund (2010, 2011), Jan-Jan Svendsen (2011, 2012), Jordi Ljungberg (2012, 2013), Jakob Nielsen (2013, 2014), Jens Peterson (2014, 2015), Erika Lindström (2015, 2016), Anders Nygaard (2016, 2017), Lars Björnsson (2017, 2018)

Question: List all employers Danny White worked for from 2012 to 2020.
Generated Answer: The New York Yankees (2012, 2014), the Los Angeles Dodgers (2015, 2016), and the Seattle Seahawks (2016, 2017).



Question: List all sports teams Roda Antar played for from 2010 to 2016.
Generated Answer: Barcelona F.C. (2010, 2011, 2012, 2014), Real Madrid C.F. (2012, 2013), Atlético Mineiro S.A. (2013, 2015)

Question: List all sports teams Edwin van der Sar, also known as Edwin van der Saar, played for from 2010 to 2016.
Generated Answer: AFC Ajax (2010, 2011, 2012), Tottenham Hots

Question: List all employers Pancracio Celdrán, also known as Pancracio Celdran, worked for from 2010 to 2018.
Generated Answer: Ayrton Senna, Universidad Católica de Madrid, Unión Europea

Question: List all positions John J. Sullivan, also known as John Joseph Sullivan, held from 2017 to 2020.
Generated Answer: Governor of New York (2017–2020), United States Representative (New York) (2016–present), United States Senator (New York) (2012–2015), United States Secretary of State (2010–2011)

Question: List all sports teams Phillip Hughes, also known as Phillip Joel Hughes, played for from 2010 to 2014.
Generated Ans

Question: List all coaches of Denmark national association football team, also known as Denmark national soccer team, from 2010 to 2020
Generated Answer: David Henriksen (2010, 2012, 2014), Lars Bjrnebäck (2012, 2013), Jens Tilsson (2013, 2014), Anders Lindegaard (2014, 2015), Johan Christensen (2015, 2016), Jordi kesson (2016, 2017), Erik Svendsen (2016, 2017), Thomas Lund (2016, 2017), Christian Klose (2016, 2017), Martin Ol

Question: List all positions Kelly Craft, also known as Kelly Knight, held from 2017 to 2020.
Generated Answer: member of the Australian House of Representatives (2017, 2018, 2019, 2020), Minister for Health and Aged Care (2018, 2019, 2020)

Question: List all heads of the government of Manila, also known as City of Manila, from 2010 to 2020 
Generated Answer: Luis Antonio Rizal (2010, 2011), Jose Maria Aquino (2011, 2012), Juan Carlos Ramos (2012, 2014), Yvonne López del Rosario (2014, 2015), José Manuel Santos (2015, 2016), Rafael P. Valdes (2016, 2017), Franc

Question: List all sports teams Marco Materazzi played for from 2010 to 2014.
Generated Answer: Milan F.C. (2010, 2011, 2012), Juventus F.C. (2011, 2012, 2013), AC Bologna (2012, 2013, 2014), Ajaccio Serie A (2013, 2014), Internazionale (2014, 2015), Roma B (2015, 2016), Udinese (2016, 2017), Atlético Madrid (2017, 2018), Real Sociedad (2018, 2019),

Question: List all coaches of Daco-Getica, also known as Juventus București, from 2015 to 2020
Generated Answer: Pompil

Question: List all chairpersons of BayernSPD, also known as SPD Bayern, from 2010 to 2020
Generated Answer: Michael Müller (2010, 2011), Thomas Schäfer (2011, 2012, 2014), Andreas Hänkel (2012, 2013), Martin Schulz (2013, 2014, 2015)

Question: List all chairpersons of Mondi Group, also known as Mondi, from 2010 to 2020
Generated Answer: Michael J. Mondi (2010, 2011, 2012, 2014, 2015, 2020)

Question: List all heads of the government of The Hague, also known as Den Haag, from 2010 to 2020 
Generated Answer: David van der

Question: List all sports teams Andrey Arshavin, also known as Andrey Sergeevich Arshavin, played for from 2010 to 2018.
Generated Answer: Chelsea F.C.

Question: List all political parties Lin Yi-shih, also known as Lin Yishi, was a member of from 2010 to 2020.
Generated Answer: Independent (2010, 2011), Democratic Progressive Party (2011, 2012, 2014, 2015, 2016, 2020)

Question: List all employers Steven P. Croley, also known as Steven Croley, worked for from 2010 to 2017.
Generated Answer: United States Department of Agriculture (USDA), National Association of State, County and Municipal Employees (NASCE), American Federation of Teachers (ATA), California Agricultural University (CAU)

Question: List all entities that owned Netscape, also known as Netscape Communications, from 2010 to 2020.
Generated Answer: Netscape Communications, Inc. (2010, 2011), AOL Corporation (2011, 2012), Microsoft Corp. (2012, 2014)

Question: List all sports teams Alou Diarra played for from 2010 to 2020.

In [12]:
# 10 shot
examples_10  = knn_search(train_questions, train_answers, test_questions, 10)
predictions_10shot_flanlarge = run_model("google/flan-t5-large", test_questions, examples_10)

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: Middlesbrough F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Republika Srpska (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018), President of Bosnia and Herzegovina (2018, 2019, 2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: List all political parties Mark Patterson, also known as Mark William James Patterson, was a member of from 2010 to 2020. A: New Zealand National Party (2010, 2011, 2012, 2013, 2014, 2015), New Zealand First (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Generated Answer: A: 

Question: List all sports teams Kostas Manolas played for from 2010 to 2020.
Generated Answer: KS Dinamo Tirana

Question: List all heads of the government of Bremen, also known as City Municipality of Bremen, from 2010 to 2020 
Generated Answer: A: Chancellor of Schleswig-Holstein

Question: List all employers Luciano Onder worked for from 2010 to 2020.
Generated Answer: BFMTV (2010, 2011), France 2 (2011, 2012, 2013, 2014, 2015), RT France (2015, 2016, 2017, 2018)

Question: List all chairpersons of Ministry of Employment and Labor, also known as MOEL, from 2010 to 2020
Generated Answer: The following is a list of the members of the Board of Trustees of the University of California, Berkeley.

Question: List all coaches of FC Petrolul Ploiești, also known as Petrolul Ploiesti, from 2010 to 2020
Generated Answer: Marius umudică (2010 – 2020)

Question: List all sports teams Jonathan Mensah played for from 2010 to 2020.
Generated Answer: A: Australian rules footballer

Question: List a

Question: List all employers Paul P. Glasziou, also known as Paul Glasziou, worked for from 2010 to 2020.
Generated Answer: Rede Bandeirantes (2010 - 2020)

Question: List all employers Rupert Sutherland worked for from 2010 to 2020.
Generated Answer: A: United States Postal Service (United States Postal Service, U.S. Postal Service, USPS)

Question: List all sports teams Andrea Pirlo, also known as Andrea Pirlo Escaleto, played for from 2010 to 2020.
Generated Answer: A: Juventus F.C.

Question: List all sports teams David Luiz, also known as David Luiz Moreira Marinho, played for from 2010 to 2020.
Generated Answer: Real Madrid CF (2010, 2011, 2012, 2013), Chelsea F.C. (2010), AS Monaco F.C. (2010, 2011

Question: List all educational institutions Evgeny Konnov attended from 2010 to 2020.
Generated Answer: A: Universidad Católica de Catalunya

Question: List all employers David Mba, also known as David U. Mba, worked for from 2010 to 2020.
Generated Answer: Washington University in S

Question: List all sports teams Mario Balotelli, also known as Mario Balotelli Barwuah, played for from 2010 to 2020.
Generated Answer: K.R.C. Genk (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020), Belgium national under-21 football team

Question: List all positions Bård Hoksrud, also known as Bård André Hoksrud, held from 2013 to 2019.
Generated Answer: A: Member of parliament (MP)

Question: List all coaches of AC Sparta Prague, also known as Sparta Prague, from 2012 to 2020
Generated Answer: Yury Maleyew (2010, 2011, 2012, 2013, 2014)

Question: List all political parties Andris Bērziņš, also known as Andris Berzins, was a member of from 2010 to 2020.
Generated Answer: A: Greek Democratic Union (Greek:   )

Question: List all chairpersons of Ministry of Internal Affairs of Ukraine, also known as Ministry of Internal Affairs, from 2010 to 2020
Generated Answer: A: Dmitry Medvedev (2012, 2014, 2015, 2016)

Question: List all heads of the government of Hamburg, also 

Question: List all sports teams Ali Karimi, also known as Mohammad Ali Karimi Pashaki, played for from 2010 to 2014.
Generated Answer: AFC Ajax (2010, 2011, 2012), SBV Vitesse (2010, 2011), Morocco national football team (2011)

Question: List all political parties Peadar Tóibín, also known as Peadar Toibin, was a member of from 2010 to 2020.
Generated Answer: Party of European Socialists (2010, 2011), Socialist Party of Hungary (2010, 2011)

Question: List all political parties Roger Holeindre was a member of from 2010 to 2020.
Generated Answer: A: Socialist Party (France)

Question: List all sports teams Cesc Fàbregas, also known as Francesc Fàbregas Soler, played for from 2010 to 2020.
Generated Answer: Real Madrid CF (2010, 2011, 2012, 2013, 2014, 2015)

Question: List all positions Alexis Tsipras, also known as Alexios Tsipras, held from 2012 to 2019.
Generated Answer: A: Member of Parliament (Municipality of Cluj-Napoca)

Question: List all employers Rachel Sheherazade, also know

...

Question: List all employers Deborah L. Birx, also known as Deborah Birx, worked for from 2010 to 2020.
Generated Answer: A: United States Postal Service (United States Postal Service), United States Postal Inspection Service (USPS), United States Department of Agriculture (USDA), United States Army (US Army), United States Air Force (USAF), United States Navy (USNA), United States Army Corps of Engineers (USACE), United States Naval Reserve (USNR), United States Marine Corps (USMC), United States Navy Reserve (USNR)

Question: List all political parties Christophe Steiner was a member of from 2010 to 2020.
Generated Answer: 2010–2020)

Question: List all employers Arti Agrawal worked for from 2011 to 2020.
Generated Answer: A: United States Postal Service (United States Postal Service), United States Postal Inspection Service (USPS), United States Department of Agriculture (USDA), United States Forest Service (USFS), United States Fish and Wildlife Service (USFWS), United States Geolo

Question: List all positions Chris Coons, also known as Senator Christopher Andrew Coons, held from 2010 to 2020.
Generated Answer: A: United States Attorney General (United States), United States Attorney for the District of Columbia

Question: List all coaches of PFC Litex Lovech from 2015 to 2020
Generated Answer: Mikhail Khodorkovsky (2015, 2015, 2016), Sergey Ivanov (2015, 2016), Alexei Ivanov (2016, 2017), Alexei Avramov (2016, 2018), Alexei Aleksandrov (2016, 2019)

Question: List all sports teams Henri Bedimo, also known as Henri Bedimo Nsame, played for from 2010 to 2020.
Generated Answer: A: AC Sparta Prague (2012, 2013, 2014), Jagiellonia Biaystok (2013, 2014), SK Slavia Prague (2014), HNK Rijeka (2015, 2016), FC Akhmat Grozny (2016, 2017, 2018, 2019), SK Sturm G

Question: List all employers Gabriel Salazar, also known as Gabriel Salazar Vergara, worked for from 2010 to 2020.
Generated Answer: A: Televisión Nacional de Chile (2010, 2011), Televisión Espaol (2010, 2011)

Que

Question: List all positions Hage Geingob, also known as Hage Gottfried Geingob, held from 2012 to 2020.
Generated Answer: European Commission for Enlargement and European Neighbourhood Policy (2010), European Commissioner for An Economy that Works for People (2010, 2011, 2012, 2013, 2014)

Question: List all political parties Dennis Hof, also known as Dennis Leroy Hof, was a member of from 2015 to 2020.
Generated Answer: Conservative Party (2010, 2011, 2012, 2013), UK Independence Party (2013, 2014)

Question: List all sports teams Charlie Adam, also known as Charles Graham Adam, played for from 2010 to 2020.
Generated Answer: Melbourne Renegades (2013), Adelaide Strikers (2013, 2014), Hobart Hurricanes (2014)

Question: List all positions Robert McClelland, also known as Robert Bruce McClelland, held from 2010 to 2012.
Generated Answer: A: Member of Parliament (MP)

Question: List all positions Michael Flynn, also known as Michael Thomas Flynn, held from 2012 to 2017.
Generated Answe

Question: List all heads of the government of North West, also known as North-West, from 2010 to 2020 
Generated Answer: Liu (2010, 2011, 2012, 2013, 2014, 2015, 2016)

Question: List all political parties Kyriakos Velopoulos, also known as Kyriakos Iosif Velopoulos, was a member of from 2010 to 2020.
Generated Answer: A: Liberal Party (UK), Labour Party (UK), Conservative Party (UK), Green Party (UK), Independence Party (UK), Independent (UK)

Question: List all sports teams Shane Long, also known as Shane Patrick Long, played for from 2010 to 2020.
Generated Answer: Mansfield Town F.C.

Question: List all sports teams Filipe Morais, also known as Filipe Alexandre Major Morais, played for from 2010 to 2020.
Generated Answer: A: Australian rules footballer

Question: List all political parties Marina Weisband was a member of from 2010 to 2020.
Generated Answer: Socialist Party (2010, 2011, 2012, 2014, 2015, 2020)

Question: List all coaches of Spain women's national association footbal

Question: List all political parties Sarath Fonseka was a member of from 2010 to 2020.
Generated Answer: Socialist Party (2014, 2015, 2016), Éc

Question: List all sports teams Adriano played for from 2010 to 2016.
Generated Answer: Paris Saint-Germain F.C.

Question: List all political parties Artur Mas was a member of from 2010 to 2020.
Generated Answer: Republican Party (2010, 2011, 2012, 2013, 2014, 2015)

Question: List all chairpersons of Green League, also known as Vihreä liitto, from 2010 to 2020
Generated Answer: 2020)

Question: List all sports teams Andrey Arshavin, also known as Andrey Sergeevich Arshavin, played for from 2010 to 2018.
Generated Answer: Chelsea F.C.

Question: List all political parties Lin Yi-shih, also known as Lin Yishi, was a member of from 2010 to 2020.
Generated Answer: Kuomintang (2010, 2020)

Question: List all employers Steven P. Croley, also known as Steven Croley, worked for from 2010 to 2017.
Generated Answer: Food and Drug Administration (2016,

In [6]:
#flan-t5-xl
all_predictions = {}
knn = [3, 5, 7]
for k in knn:
    print(f"K = {k}")
    examples_flanxl  = knn_search(train_questions, train_answers, test_questions, k)
    predictions_flanxl = run_model("google/flan-t5-xl", test_questions, examples_flanxl)
    all_predictions[k] = predictions_flanxl
    #EM
    EM_flanxl= EM(predictions_flanxl, true_answers)
    print(f"Exact Match: {EM_flanxl:.2f}")

    #F1 score
    f1_score_flanxl= f1_score(predictions_flanxl, true_answers)
    print(f"F1 Score: {f1_score_flanxl}")



K = 3


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: United States men's national basketball team (2010, 2011)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: Prime Minister of Ukraine (2010–2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: Fianna Fáil (2010, 2011, 2012), Sinn Féin (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Generated Answer: Liverpool F.C.

Question: List all heads of the government of Newfoundland and Labrador, also known as NL, from 2013 to 2020 
Generated Answer: Stephen McNeil (2013, 2014, 2015, 2016, 2017, 2018), John McCallum (2018, 2019, 2020)

Question: List all employers Samy Azer, also known as Sam

Question: List all chairpersons of Ministry of Employment and Labor, also known as MOEL, from 2010 to 2020
Generated Answer: Yves Leterme (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)

Question: List all coaches of FC Petrolul Ploiești, also known as Petrolul Ploiesti, from 2010 to 2020
Generated Answer: Petrolul Ploiești (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017)

Question: List all sports teams Jonathan Mensah played for from 2010 to 2020.
Generated Answer: Ghana national football team (2010, 2011, 2012), Sunderland A.F.C. (2010, 2011), Stade Rennais F.C. (2012, 2013), Accra Hearts of Oak SC (2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020), FC Nitra (2013, 2014), Asante Kotoko F.C. (2013), AFC Eskilstuna (2016, 2017, 2018, 2019, 2020)

Question: List all employers Angela Casini worked for from 2010 to 2020.
Generated Answer: United States Department of Agriculture (USDA) (2010, 2011, 2012, 2013, 2014, 2015, 2016), United States Department of Health and Huma

Question: List all positions Arayik Harutyunyan, also known as Arayik Vladimiri Harutyunyan, held from 2010 to 2020.
Generated Answer: Prime Minister of Armenia (2010–2020)

Question: List all coaches of NK Domžale from 2016 to 2020
Generated Answer: Tomislav Ivkovi (2016), Goran Tomi (2017, 2018, 2019, 2020)

Question: List all heads of the government of Nantes, also known as Naoned, from 2012 to 2020 
Generated Answer: Jean-Marc Ayrault (2012–2020)

Question: List all chairpersons of International Olympic Committee, also known as IOC, from 2010 to 2020
Generated Answer: IOC President Thomas Bach (2010–2020)

Question: List all coaches of FC Botoșani, also known as Fotbal Club Botoșani, from 2014 to 2020
Generated Answer: tefan igănescu (2014, 2015, 2016, 2017), tefan Ionescu (2017, 2018), tefan Vukcevic (2019, 2020)

Question: List all coaches of Brighton & Hove Albion F.C., also known as Brighton and Hove Albion F.C., from 2014 to 2020
Generated Answer: Chris Hughton (2014–2019)

Qu

...

Question: List all political parties Mircea Geoană was a member of from 2010 to 2020.
Generated Answer: Democratic Liberal Party (2010, 2011, 2012, 2013, 2014, 2015), National Liberal Party (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of FC Smolevichi, also known as FC Smolevichi-STI, from 2011 to 2020
Generated Answer: Aleksandr Brazevich (2011, 2012, 2013), Mikalay Ryndzyuk (2015, 2016, 2017), Alyaksey Dvarnyuk (2019), Vyacheslav Geraschenko (2020)

Question: List all positions Mark Francois, also known as Mark Gino Francois, held from 2010 to 2020.
Generated Answer: Member of the National Assembly of Quebec (2010, 2011, 2012, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all heads of the government of New Jersey, also known as Garden State, from 2010 to 2020 
Generated Answer: Chris Christie (2010, 2011, 2012, 2013), Phil Murphy (2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Kimberly Guilfoyle, also known as Kimberly Ann Guil

Question: List all positions David Joyce, also known as David Patrick Joyce, held from 2010 to 2019.
Generated Answer: Member of the Australian House of Representatives (2010–2019)

Question: List all positions Mark Sedwill, also known as Mark Philip Sedwill, held from 2017 to 2020.
Generated Answer: Secretary of State for Culture, Media and Sport (2017–2020)

Question: List all employers Frances Spalding worked for from 2010 to 2020.
Generated Answer: University College London (2010, 2011), University College London (2011, 2012), University College London (2012), University College London (2013), University College London (2014), University College London (2015), University College London (2016), University College London (2017), University College London (2018), University College London (2019, 2020)

Question: List all coaches of TSV Hartberg from 2012 to 2020
Generated Answer: Patrik Schmid (2012–2013, 2013–2014, 2014–2015, 2015–2016, 2016–2017, 2017–2018, 2018–2019, 2019–2020)

Qu

Question: List all coaches of K.A.S. Eupen from 2017 to 2020
Generated Answer: Jürgen Melzer (2017, 2018), Jürgen Moltke (2018, 2019), Jürgen Meltzer (2019, 2020)

Question: List all employers Farida Kabir, also known as Farida Mohammad Kabir, worked for from 2014 to 2019.
Generated Answer: Farida Mohammad Kabir (2014-2019)

Question: List all educational institutions Ashley Hoffman, also known as Ashley Joy Hoffman, attended from 2011 to 2018.
Generated Answer: University of California, Los Angeles (UCLA), University of California, Berkeley (UCB), University of Southern California (USC)

Question: List all positions Nguyễn Phú Trọng, also known as Nguyễn Phú Trọng, held from 2010 to 2020.
Generated Answer: Prime Minister of the People's Republic of Vietnam (2010–2020)

Question: List all sports teams Craig Bellamy, also known as Craig Douglas Bellamy, played for from 2010 to 2014.
Generated Answer: Cardiff City F.C.

Question: List all employers Wendy L. Queen, also known as Wendy Lee

Question: List all positions Rachel Treweek, also known as Rachel Montgomery, held from 2010 to 2020.
Generated Answer: Member of the New South Wales Legislative Assembly (2010–2020)

Question: List all sports teams Cristiano Ronaldo, also known as Cristiano Ronaldo dos Santos Aveiro, played for from 2010 to 2020.
Generated Answer: Real Madrid C.F.

Question: List all positions Jo Johnson, also known as Joseph Johnson, held from 2018 to 2018.
Generated Answer: Member of Parliament (MP)

Question: List all sports teams Rodrigo Palacio, also known as Rodrigo Sebastián Palacio Alcalde, played for from 2010 to 2020.
Generated Answer: Universidad Católica (2010, 2011), Universidad Nacional (2011, 2012), Universidad Autónoma de México (2012, 2013), Universidad de Buenos Aires (2013, 2014), Universidad Complutense (2014, 2015)

Question: List all coaches of F.C. Nantes, also known as FC Nantes, from 2016 to 2020
Generated Answer: Philippe Montanier (2016–2020)

Question: List all sports teams

NameError: name 'EM' is not defined

In [11]:
knn = [5, 7]
for k in knn:
    print(f"K = {k}")
    examples_flanxl  = knn_search(train_questions, train_answers, test_questions, k)
    predictions_flanxl = run_model("google/flan-t5-xl", test_questions, examples_flanxl)
    all_predictions[k] = predictions_flanxl
    #EM
    EM_flanxl= EM(predictions_flanxl, true_answers)
    print(f"Exact Match: {EM_flanxl:.2f}")

    #F1 score
    f1_score_flanxl= f1_score(predictions_flanxl, true_answers)
    print(f"F1 Score: {f1_score_flanxl}")


K = 5


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: United States men's national basketball team (2010, 2011)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: Prime Minister of Ukraine (2010–2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: European Parliament (2011, 2012, 2013, 2014), Teachta Dála (2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.
Generated Answer: Liverpool F.C.

Question: List all heads of the government of Newfoundland and Labrador, also known as NL, from 2013 to 2020 
Generated Answer: Stephen McNeil (2013, 2014, 2015, 2016, 2017, 2018), John McCallum (2018, 2019, 2020)

Question: List all employers Samy Azer, also k

Question: List all heads of the government of Bremen, also known as City Municipality of Bremen, from 2010 to 2020 
Generated Answer: Klaus Wowereit (2010, 2011, 2012, 2013, 2014), Michael Müller (2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all employers Luciano Onder worked for from 2010 to 2020.
Generated Answer: Universidad Nacional Autónoma de México (2010, 2011, 2012, 2013, 2014, 2015, 2016), Universidad Católica de Chile (2011, 2012, 2013, 2015, 2016, 2017)

Question: List all chairpersons of Ministry of Employment and Labor, also known as MOEL, from 2010 to 2020
Generated Answer: Yves Leterme (2010, 2011, 2012, 2014, 2015, 2016), Yves-Marie Leterme (2014, 2015, 2016, 2017), Yvonne Leterme (2019, 2020)

Question: List all coaches of FC Petrolul Ploiești, also known as Petrolul Ploiesti, from 2010 to 2020
Generated Answer: Petrolul Ploiești (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Jonathan Mensah played for f

Question: List all coaches of NK Domžale from 2016 to 2020
Generated Answer: Tomislav Ivkovi (2016), Goran Tomi (2017, 2018, 2019, 2020)

Question: List all heads of the government of Nantes, also known as Naoned, from 2012 to 2020 
Generated Answer: Jean-Marc Ayrault (2012, 2014, 2015, 2016, 2017, 2018), Nathalie Appéré (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all chairpersons of International Olympic Committee, also known as IOC, from 2010 to 2020
Generated Answer: Sepp Blatter (2010–2020)

Question: List all coaches of FC Botoșani, also known as Fotbal Club Botoșani, from 2014 to 2020
Generated Answer: tefan igănescu (2014, 2015, 2016, 2017), tefan Ioniţă (2015, 2016, 2017, 2018), tefan Gheorghiu (2019, 2020)

Question: List all coaches of Brighton & Hove Albion F.C., also known as Brighton and Hove Albion F.C., from 2014 to 2020
Generated Answer: Chris Hughton (2014, 2015, 2016, 2017, 2018), Paul Clement (2019, 2020)

Question: List all positions Elio Di Rupo held from 

...

Question: List all chairpersons of Social Democratic Party, also known as Partidul Social Democrat, from 2010 to 2020
Generated Answer: A: Ion Iliescu (2010, 2012, 2014, 2015, 2016)

Question: List all heads of the government of Nizhny Novgorod Oblast, also known as Gorky Oblast (USSR), from 2010 to 2020 
Generated Answer: Ivan Karnilin (2015, 2016, 2017), Vladimir Panov (2018, 2019, 2020)

Question: List all chairpersons of German Economic Association from 2010 to 2020
Generated Answer: Wolfgang Schäuble (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all coaches of New Zealand national rugby union team, also known as All Blacks, from 2010 to 2020
Generated Answer: Steve Hansen (2010, 2011), Stuart Lancaster (2011, 2012, 2013, 2014, 2015), Eddie Jones (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all political parties Rathika Sitsabaiesan was a member of from 2010 to 2020.
Generated Answer: Tamil National Alliance (TNA)

Question: List all cha

Question: List all heads of the government of New Jersey, also known as Garden State, from 2010 to 2020 
Generated Answer: Chris Christie (2010, 2011, 2012, 2013), Phil Murphy (2014, 2015, 2016, 2017, 2018, 2019), Phil Murphy (2019, 2020)

Question: List all employers Kimberly Guilfoyle, also known as Kimberly Ann Guilfoyle, worked for from 2010 to 2020.
Generated Answer: UnitedHealth Group (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)

Question: List all positions Chris Coons, also known as Senator Christopher Andrew Coons, held from 2010 to 2020.
Generated Answer: United States senator

Question: List all coaches of PFC Litex Lovech from 2015 to 2020
Generated Answer: Hristo Kolev (2014, 2015, 2016), Eduard Eranosyan (2016, 2017), Bruno Akrapovi (2017, 2018, 2019, 2020)

Question: List all sports teams Henri Bedimo, also known as Henri Bedimo Nsame, played for from 2010 to 2020.
Generated Answer: HNK Rijeka (2015, 2016), FC Akhmat Grozny (2016, 2017, 2018, 2019), SK St

Question: List all sports teams Charlie Adam, also known as Charles Graham Adam, played for from 2010 to 2020.
Generated Answer: Melbourne Renegades (2013), Adelaide Strikers (2013, 2014), Hobart Hurricanes (2014, 2015)

Question: List all positions Robert McClelland, also known as Robert Bruce McClelland, held from 2010 to 2012.
Generated Answer: Governor of New York (2010–2012)

Question: List all positions Michael Flynn, also known as Michael Thomas Flynn, held from 2012 to 2017.
Generated Answer: Secretary of State for Health (2012–2017), Secretary of State for Education (2012–2017)

Question: List all sports teams Lomana LuaLua played for from 2010 to 2020.
Generated Answer: Borussia Mönchengladbach

Question: List all entities that owned Culpeper Star-Exponent, also known as Culpeper Star Exponent, from 2015 to 2020.
Generated Answer: Culpeper Star-Exponent, Inc.

Question: List all sports teams Aiden McGeady, also known as Aiden John McGeady, played for from 2010 to 2020.
Genera

Question: List all positions Grete Faremo held from 2010 to 2013.
Generated Answer: Minister of Health (2010, 2011), Minister of Labour (2011, 2012), Minister of Health (2012, 2013)

Question: List all heads of the government of Rome, also known as The Eternal City, from 2010 to 2020 
Generated Answer: Matteo Renzi (2010, 2011, 2012, 2013, 2014), Dario Nardella (2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all heads of the government of North West, also known as North-West, from 2010 to 2020 
Generated Answer: John Swinney (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017), John Swinney (2017), John Swinney (2018)

Question: List all political parties Kyriakos Velopoulos, also known as Kyriakos Iosif Velopoulos, was a member of from 2010 to 2020.
Generated Answer: SYRIZA (2010, 2011, 2012, 2013, 2014, 2015), Popular Unity (Greece) (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Shane Long, also known as Shane Patrick Long, played for from 2010 to 2020.


Question: List all coaches of Inteja-Dominican, also known as Inteja-MMR Dominican, from 2015 to 2020
Generated Answer: António Conceiço da Silva Oliveira (2019, 2020)

Question: List all sports teams Jonny Howson, also known as Jonathan Mark Howson, played for from 2010 to 2020.
Generated Answer: Leeds United F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019)

Question: List all entities that owned MyAnimeList, also known as MyAnimeList, LLC, from 2010 to 2019.
Generated Answer: MyAnimeList, LLC (2010–2019)

Question: List all political parties Sarath Fonseka was a member of from 2010 to 2020.
Generated Answer: Sri Lanka Freedom Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018), United National Party (2016, 2017, 2018), United People's Freedom Alliance (2018, 2019, 2020)

Question: List all sports teams Adriano played for from 2010 to 2016.
Generated Answer: France national under-21 association football team

Question: List all political parties Artur Mas was

In [16]:
knn = [10]
for k in knn:
    print(f"K = {k}")
    examples_flanxl  = knn_search(train_questions, train_answers, test_questions, k)
    predictions_flanxl = run_model("google/flan-t5-xl", test_questions, examples_flanxl)
    all_predictions[k] = predictions_flanxl
    #EM
    EM_flanxl= EM(predictions_flanxl, true_answers)
    print(f"Exact Match: {EM_flanxl:.2f}")

    #F1 score
    f1_score_flanxl= f1_score(predictions_flanxl, true_answers)
    print(f"F1 Score: {f1_score_flanxl}")


K = 10


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Question: List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.
Generated Answer: Middlesbrough F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020), England national under-19 association football team (2011), Darlington F.C. (2011)

Question: List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.
Generated Answer: President of Republika Srpska (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018), Chairman of the Presidency of Bosnia and Herzegovina (2018, 2019, 2020)

Question: List all political parties Paul Murphy was a member of from 2010 to 2020.
Generated Answer: A: List all political parties Jeremy Corbyn was a member of from 2010 to 2020. A: Democratic Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)

Question: List all sports teams Andy Carroll, also known as Andrew Thomas Carroll, played for from 2010 to 2020.

Question: List all sports teams Kostas Manolas played for from 2010 to 2020.
Generated Answer: Albania national association football team (2010, 2011, 2012, 2013), K.F. Tirana (2010, 2015, 2016, 2017, 2018, 2019, 2020), KF Sknderbeu Korç (2011, 2012, 2013, 2014), Flamurtari

Question: List all heads of the government of Bremen, also known as City Municipality of Bremen, from 2010 to 2020 
Generated Answer: A: Hans-Gerhard Schroeder

Question: List all employers Luciano Onder worked for from 2010 to 2020.
Generated Answer: BFMTV (2010, 2011), France 2 (2011, 2012, 2013, 2014, 2015), RT France (2015, 2016, 2017)

Question: List all chairpersons of Ministry of Employment and Labor, also known as MOEL, from 2010 to 2020
Generated Answer: A: Presidents of the United States from 2010 to 2020 A: George W. Bush (2009-2012), Barack Obama (2011-2012), Hillary Clinton (2012-2016), Barack Obama (2016-present), Donald Trump (2017-present), Mike Pence (2017-present)

Question: List all coaches of FC

Question: List all coaches of FC Botoșani, also known as Fotbal Club Botoșani, from 2014 to 2020
Generated Answer: A: José Luis Rodrguez lvarez

Question: List all coaches of Brighton & Hove Albion F.C., also known as Brighton and Hove Albion F.C., from 2014 to 2020
Generated Answer: Chris Hughton (2005–2006), Steve Bruce (2006–2007), Steve Bruce (2007–2009), Steve Bruce (2009–2011), Steve Bruce (2011–2012), Steve Bruce (2012–2013), Steve Bruce (2013–2014), Steve Bruce (2014–2015), Steve Bruce (2015–2016), Steve Bruce (2016–2017), Steve Bruce (2017–2018), Steve Bruce (2018–2019), Steve Bruce (2019–2020)

Question: List all positions Elio Di Rupo held from 2010 to 2020.
Generated Answer: 2020)

Question: List all employers Paul P. Glasziou, also known as Paul Glasziou, worked for from 2010 to 2020.
Generated Answer: Rede Bandeirantes (2010-2020)

Question: List all employers Rupert Sutherland worked for from 2010 to 2020.
Generated Answer: A: United States Department of Agriculture (USD

...

Question: List all chairpersons of German Economic Association from 2010 to 2020
Generated Answer: Bronislovas Lubys (2010, 2011), Robertas Dargis (2012, 2013, 2014, 2015)

Question: List all coaches of New Zealand national rugby union team, also known as All Blacks, from 2010 to 2020
Generated Answer: A: Uli Hoeneß (2002–03), Uli Hoenéß (2003–05), Uli Hoheneß (2005–06), Uli Hoehneß (2006–07), Uli Hohmann (2007–08), Uli Hohneß (2008–09), Uli Hoheß (2009–10), Uli Hoheßen (2010–11)

Question: List all political parties Rathika Sitsabaiesan was a member of from 2010 to 2020.
Generated Answer: Q: List all political parties Vasile Blaga was a member of from 2010 to 2020. A: Democratic Liberal Party (2010, 2011, 2012, 2013, 2014, 2015), National Liberal Party (2015, 2016, 2017, 2018, 2019, 2020)

Question: List all chairpersons of CRH, also known as CRH p.l.c., from 2010 to 2020
Generated Answer: Kirsan Ilyumzhinov (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018)

Question: List all po

Question: List all heads of the government of New Jersey, also known as Garden State, from 2010 to 2020 
Generated Answer: Peter Shumlin (2011–2020)

Question: List all employers Kimberly Guilfoyle, also known as Kimberly Ann Guilfoyle, worked for from 2010 to 2020.
Generated Answer: A: Rebecca Lingwood, worked for from 2010 to 2020. A: University of Oxford (2010, 2011, 2012), University College London (2017, 2018, 2019, 2020)

Question: List all positions Chris Coons, also known as Senator Christopher Andrew Coons, held from 2010 to 2020.
Generated Answer: A: Attorney General of the United States

Question: List all coaches of PFC Litex Lovech from 2015 to 2020
Generated Answer: Vyacheslav Geraschenko (2015, 2016, 2017), Vyacheslav Cherniavtsev (2019, 2020)

Question: List all sports teams Henri Bedimo, also known as Henri Bedimo Nsame, played for from 2010 to 2020.
Generated Answer: HNK Rijeka (2015, 2016), FC Akhmat Grozny (2016, 2017, 2018, 2019), SK Sturm G

Question: List all emp

Question: List all sports teams Lomana LuaLua played for from 2010 to 2020.
Generated Answer: Borussia Mönchengladbach

Question: List all entities that owned Culpeper Star-Exponent, also known as Culpeper Star Exponent, from 2015 to 2020.
Generated Answer: BC Partners

Question: List all sports teams Aiden McGeady, also known as Aiden John McGeady, played for from 2010 to 2020.
Generated Answer: A: Dempsey

Question: List all coaches of NK Osijek, also known as Nogometni klub Osijek, from 2012 to 2020
Generated Answer: A: List of United States presidential candidates in 2016 A: Donald Trump, Hillary Clinton, Donald Trump Jr., Mike Pence, Mike Pence Jr., Marco Rubio, Rick Santorum, Marco Rubio

Question: List all educational institutions Clayton Custer attended from 2010 to 2020.
Generated Answer: College of the Ozarks (2010, 2011), University of Missouri-Kansas City (2011, 2012), University of Kansas (2012, 2013), University of Oklahoma (2013, 2014, 2015, 2016)

Question: List all pos

Question: List all entities that owned Financial Times, also known as The Financial Times, from 2010 to 2020.
Generated Answer: A: Vivendi SA

Question: List all positions Kevin Rudd, also known as Kevin Michael Rudd, held from 2010 to 2013.
Generated Answer: A: Julian Smith, held from 2010 to 2020. A: Minister for Agriculture, Food and the Marine (2010, 2011), Minister for Justice (2010, 2011)

Question: List all heads of the government of Basse-Terre, also known as Basse Terre, from 2010 to 2020 
Generated Answer: A: Jean-Baptiste Lefebvre

Question: List all sports teams Marco Materazzi played for from 2010 to 2014.
Generated Answer: Arsenal F.C.)

Question: List all coaches of Daco-Getica, also known as Juventus București, from 2015 to 2020
Generated Answer: Pompil

Question: List all chairpersons of BayernSPD, also known as SPD Bayern, from 2010 to 2020
Generated Answer: Yuriy Lutsenko (2014, 2015), Vitali Klitschk (2019, 2020)

Question: List all chairpersons of Mondi Group, also

Question: List all employers Steven P. Croley, also known as Steven Croley, worked for from 2010 to 2017.
Generated Answer: Verily Life Sciences

Question: List all entities that owned Netscape, also known as Netscape Communications, from 2010 to 2020.
Generated Answer: A: National Geographic Society

Question: List all sports teams Alou Diarra played for from 2010 to 2020.
Generated Answer: A: Brazil national under-20 football team (2010, 2011), Brazil national under-21 football team (2011, 2012), Brazil national under-23 football team (2012, 2013), Brazil national under-17 football team (2013, 2014), Brazil national under-18 football team (2014, 2015), Brazil national under-19 football team (2015, 2016)

Question: List all entities that owned HMS Protector, also known as MV Polarbjørn, from 2010 to 2020.
Generated Answer: A: Royal Navy

Question: List all positions David Lidington, also known as David Roy Lidington, held from 2010 to 2019.
Generated Answer: A: Secretary of State for 

In [ ]:
# LLama2 and Mistral model
